In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [5]:
def get_historical_data(tickers, start_date, end_date):
    """
    Retrieve historical stock prices for a list of tickers.
    
    Args:
    - tickers (list): List of stock tickers (e.g., ["AAPL", "MSFT"]).
    - start_date (str): Start date in the format "YYYY-MM-DD".
    - end_date (str): End date in the format "YYYY-MM-DD".
    
    Returns:
    - DataFrame: DataFrame containing historical stock prices for the specified tickers and date range.
    """
    # Create an empty list to store the historical dataframes
    historical_data = []
    
    # Iterate over each ticker and fetch historical data
    for ticker in tickers:
        # Fetch historical data from Yahoo Finance
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        
        # Add a column to identify the stock ticker
        stock_data["Ticker"] = ticker
        
        # Append the data to the historical_data list
        historical_data.append(stock_data)
    
    # Concatenate the dataframes in the list into a single dataframe
    combined_data = pd.concat(historical_data)
    
    return combined_data

# Example usage
tickers = ["AAPL", "MSFT", "MGC=F"]
start_date = "2020-01-01"
end_date = "2021-01-01"
historical_prices = get_historical_data(tickers, start_date, end_date)

# Display the first few rows of the DataFrame
print(historical_prices.head())



[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2020-01-02  74.059998  75.150002  73.797501  75.087502  73.059425  135480400   
2020-01-03  74.287498  75.144997  74.125000  74.357498  72.349136  146322800   
2020-01-06  73.447502  74.989998  73.187500  74.949997  72.925636  118387200   
2020-01-07  74.959999  75.224998  74.370003  74.597504  72.582649  108872000   
2020-01-08  74.290001  76.110001  74.290001  75.797501  73.750259  132079200   

           Ticker  
Date               
2020-01-02   AAPL  
2020-01-03   AAPL  
2020-01-06   AAPL  
2020-01-07   AAPL  
2020-01-08   AAPL  


In [2]:
class Portfolio:
    def __init__(self, tickers, start_date, end_date, initial_weights=None):
        """
        Initialize a portfolio with given assets and weights.
        
        Args:
        - tickers (list): List of stock tickers (e.g., ["AAPL", "MSFT"]).
        - start_date (str): Start date in the format "YYYY-MM-DD".
        - end_date (str): End date in the format "YYYY-MM-DD".
        - initial_weights (dict, optional): Initial weights for each asset (default: equal weights).
        """
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        
        # Initialize portfolio weights
        if initial_weights is None:
            self.weights = {ticker: 1/len(tickers) for ticker in tickers}
        else:
            self.weights = initial_weights
        
        # Retrieve historical data
        self.historical_data = self._retrieve_historical_data()
        
        # Calculate daily returns
        self.daily_returns = self._calculate_daily_returns()
        
        # Calculate portfolio value
        self.portfolio_value = self._calculate_portfolio_value()
        
        # Calculate portfolio statistics
        self.statistics = self._calculate_statistics()
    
    def _retrieve_historical_data(self):
        """Retrieve historical stock prices for the portfolio assets."""
        # Fetch historical data for the tickers
        data = yf.download(self.tickers, start=self.start_date, end=self.end_date)
        
        # Drop any NaN values
        data.dropna(inplace=True)
        
        # Return the DataFrame
        return data
    
    def _calculate_daily_returns(self):
        """Calculate daily returns for each asset."""
        # Calculate daily percentage change in prices
        daily_returns = self.historical_data['Adj Close'].pct_change()

        # Drop any NaN values
        self.historical_data.dropna(inplace=True)
        
        return daily_returns
    
    def _calculate_portfolio_value(self):
        """Calculate the portfolio value over time based on historical data and weights."""
        # Calculate daily portfolio values
        daily_portfolio_values = (self.historical_data['Adj Close'] * pd.Series(self.weights)).sum(axis=1)
        
        return daily_portfolio_values
    
    def _calculate_statistics(self):
        """Calculate portfolio statistics such as total return, volatility, and Sharpe ratio."""
        # Calculate total return
        total_return = (self.portfolio_value.iloc[-1] / self.portfolio_value.iloc[0]) - 1
        
        # Calculate annualized volatility (standard deviation of daily returns)
        annualized_volatility = self.daily_returns.std() * np.sqrt(252)
        
        # Calculate annualized Sharpe ratio
        risk_free_rate = 0.02  # Example risk-free rate
        excess_returns = self.daily_returns - risk_free_rate / 252
        annualized_sharpe_ratio = excess_returns.mean() / excess_returns.std() * np.sqrt(252)
        
        statistics = {
            "Total Return": total_return,
            "Annualized Volatility": annualized_volatility,
            "Annualized Sharpe Ratio": annualized_sharpe_ratio
        }
        
        return statistics

    def mean_variance_optimization(self, target_return=None):
        """Perform mean-variance optimization to find optimal portfolio weights."""
        # Define objective function
        def objective(weights):
            portfolio_return = (self.daily_returns * weights).sum(axis=1).mean() * 252
            portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(self.daily_returns.cov() * 252, weights)))
            if target_return is None:
                return -portfolio_return / portfolio_volatility  # Minimize negative Sharpe ratio
            else:
                return (portfolio_return - target_return) ** 2  # Minimize deviation from target return
        
        # Define constraints
        constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
        
        # Define bounds (optional)
        bounds = [(0, 1) for _ in range(len(self.tickers))]  # Bounds for weights between 0 and 1
        
        # Initial guess (optional)
        if self.weights is not None:
            initial_guess = list(self.weights.values())
        else:
            initial_guess = [1/len(self.tickers) for _ in range(len(self.tickers))]
        
        # Perform optimization
        result = minimize(objective, initial_guess, method='SLSQP', constraints=constraints, bounds=bounds)
        
        if result.success:
            optimal_weights = {ticker: weight for ticker, weight in zip(self.tickers, result.x)}
            return optimal_weights
        else:
            raise ValueError("Optimization failed.")

tickers = ["AAPL", "MSFT", "GOOGL"]
start_date = "2021-01-01"
end_date = "2022-01-01"
portfolio = Portfolio(tickers, start_date, end_date)

# Perform mean-variance optimization without specifying target return
optimal_weights = portfolio.mean_variance_optimization()
print("Optimal Weights (without target return):", optimal_weights)

# Perform mean-variance optimization with a target return
target_return = 0.1  # Example target return (10%)
optimal_weights_target = portfolio.mean_variance_optimization(target_return=target_return)
print("Optimal Weights (with target return):", optimal_weights_target)


c:\Users\willb\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\willb\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\willb\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  3 of 3 completed

Optimal Weights (without target return): {'AAPL': 6.25964416140483e-18, 'MSFT': 0.4850532051136973, 'GOOGL': 0.5149467948863027}
Optimal Weights (with target return): {'AAPL': 0.9999999999999997, 'MSFT': 5.399326818977812e-17, 'GOOGL': 3.365363543395006e-16}


In [9]:
portfolio.tickers

NameError: name 'portfolio' is not defined